<a href="https://colab.research.google.com/github/GeorgeChieffi/CNN_CIFAR10/blob/main/Proj1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torchvision
import torch
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import os

# Create an 'output' directory if not already exists
outdir = 'output'
os.makedirs(outdir, exist_ok=True)

# Load CIFAR-10 training set
data_train = torchvision.datasets.CIFAR10('./content/', download=True, train=True)
data_test = torchvision.datasets.CIFAR10('./content/', download=True, train=False)

# Convert the data to numpy arrays
x_train_total = np.array(data_train.data)
label_train_total = np.array(data_train.targets)

split_index = 5000
x_train_np = x_train_total[split_index:]
label_train_np = label_train_total[split_index:]



# Validation_Set
x_validation_np = x_train_total[:split_index]
label_validation_np = label_train_total[:split_index]



x_test_np = np.array(data_test.data)
label_test_np = np.array(data_test.targets)

# Find an accelerator
if torch.cuda.is_available():
    my_device = torch.device("cuda")
elif torch.backends.mps.is_available():
    torch.backends.mps.is_built()
    my_device = torch.device("mps")
else:
    my_device = torch.device("cpu")
print("Running on: ",my_device)

# Data normalization
x_train_np = x_train_np / 255.0
x_validation_np = x_validation_np / 255.0
x_test_np = x_test_np / 255.0


# Convert to tensors
x_train     = torch.tensor(x_train_np, requires_grad=False, device=my_device, dtype=torch.float32)
label_train = torch.tensor(label_train_np, requires_grad=False, device=my_device)
x_validation     = torch.tensor(x_validation_np, requires_grad=False, device=my_device, dtype=torch.float32)
label_validation = torch.tensor(label_validation_np, requires_grad=False, device=my_device)

x_test     = torch.tensor(x_test_np, requires_grad=False, device=my_device, dtype=torch.float32)
label_test = torch.tensor(label_test_np, requires_grad=False, device=my_device)

#Set
n_train = x_train.shape[0]
n_test  = x_test.shape[0]
n_validation = x_validation.shape[0]

sY = x_train.shape[1]
sX = x_train.shape[2]
n_class = 10
chan  = 3


# Reshape the data
x_train = torch.permute(x_train, (0,3,1,2))
y_train = F.one_hot(label_train, n_class).type(torch.float32)

x_validation = torch.permute(x_validation, (0,3,1,2))
y_validation = F.one_hot(label_validation, n_class).type(torch.float32)

x_test = torch.permute(x_test, (0,3,1,2))
y_test = F.one_hot(label_test, n_class).type(torch.float32)

# -----
# Create Neural Network
# -----
class ResNet(nn.Module):
  def __init__(self):
    super(ResNet, self).__init__()
    filters = 16
    # start of first block
    # nn.Conv2d(in_channels, out_channels, kernel_size, padding=1)
    self.first_conv2d = nn.Conv2d(3,16,3,1,1)
    self.bn1 = nn.BatchNorm2d(filters)
    self.second_conv2d = nn.Conv2d(16,16,3,1,1)
    self.bn2 = nn.BatchNorm2d(filters)

    # start of second block
    self.third_conv2d = nn.Conv2d(16,16,3,1,1)
    self.bn3 = nn.BatchNorm2d(filters)
    self.fourth_conv2d = nn.Conv2d(16,16,3,1,1)
    self.bn4 = nn.BatchNorm2d(filters)

    # start of third block
    self.fifth_conv2d = nn.Conv2d(16,16,3,1,1)
    self.bn5 = nn.BatchNorm2d(filters)
    self.sixth_conv2d = nn.Conv2d(16,16,3,1,1)
    self.bn6 = nn.BatchNorm2d(filters)

    self.avg_pool = nn.AvgPool2d(kernel_size=2, stride=2)
    self.first_linear = nn.Linear(1024, 512)
    self.second_linear = nn.Linear(512, 10)

  def forward(self,x):
    batch_size = x.shape[0]
    channels = x.shape[1]
    rows = x.shape[2]
    cols = x.shape[3]

    # Start first block
    a = self.first_conv2d(x)
    a = self.bn1(a)
    a = F.leaky_relu_(a)
    a = self.second_conv2d(a)
    a = self.bn2(a)
    a[:, 0:3, :, :] = a[:, 0:3, :, :] + x # Skip Connection
    a = F.leaky_relu_(a)
    a = self.avg_pool(a)
    # End first block

    # Start second block
    b = self.third_conv2d(a)
    b = self.bn3(b)
    b = F.leaky_relu_(b)
    b = self.fourth_conv2d(b)
    b = self.bn4(b)
    b[:, 0:16, :, :] = b[:, 0:16, :, :] + a # Skip Connection
    b = F.leaky_relu_(b)
    b = self.avg_pool(b)
    # End second block

    # Start third block
    c = self.fifth_conv2d(b)
    c = self.bn5(c)
    c = F.leaky_relu_(c)
    c = self.sixth_conv2d(c)
    c = self.bn6(c)
    c[:, 0:16, :, :] = c[:, 0:16, :, :] + b # Skip Connection
    d = F.leaky_relu_(c)
    # End third block

    # Reshape before last layers
    d = d.view(batch_size, -1)

    d = self.first_linear(d)
    d = F.leaky_relu_(d)
    d = self.second_linear(d)
    d = F.softmax(d,dim=1)
    return d

# -----
# Create model object
# -----
model = ResNet().to(my_device)

# -----
# Train the model
# -----

# hyperparameters
batch_size = 100
n_epoch = 100
n_train = x_train.data.shape[0]
n_batch_train = n_train // batch_size
n_batch_validation = n_validation // batch_size
n_batch_test = n_test // batch_size
learningRate = 0.01
prev_validation_loss = 0.0

# create and optimizer
optim = torch.optim.SGD(model.parameters(), lr = learningRate)

train_loss_coords = []
train_acc_coords = []

validation_loss_coords = []
validation_acc_coords = []

test_loss_coords = []
test_acc_coords = []


for epoch in range(n_epoch):
  print('-----')
  train_epoch_loss = 0.0
  train_total = 0
  train_correct = 0

  for batch in range(n_batch_train):
    # Reset optimizer for gradient descent
    optim.zero_grad()

    # Start / end indicies of the data
    sidx = batch * batch_size
    eidx = (batch+1) * batch_size

    # Grab the data and the labels for the batch
    X = x_train[sidx:eidx]
    Y = y_train[sidx:eidx]

    # Run the model
    Yhat = model(X)

    # Loss
    loss = -(torch.sum(Y*torch.log(Yhat+1e-15))/batch_size)

    # Gradient descent
    loss.backward()
    optim.step()

    # Keep track of the loss !!!
    train_loss_np = loss.detach().cpu().numpy()
    train_epoch_loss = train_epoch_loss + train_loss_np

    # Accuracy
    _, predicted = torch.max(Yhat.data,1)
    _, Y_train = torch.max(Y,1)

    train_total += Y_train.size(0)
    train_correct += (predicted == Y_train).sum().item()

  train_epoch_loss = train_epoch_loss / n_train
  train_accuracy = (train_correct / (n_batch_train * batch_size)) * 100
  print(f'Training: Epoch {epoch+1}/{n_epoch} {train_epoch_loss:.5f} Loss, {train_accuracy:.2f}% Accurate')
  train_loss_coords.append(train_epoch_loss)
  train_acc_coords.append(train_accuracy)

  # Validation set
  with torch.no_grad():
    new_validation_loss = 0.0
    validation_total = 0
    validation_correct = 0
    for batch in range(n_batch_validation):
      # Start / end indicies of the data
      sidx = batch * batch_size
      eidx = (batch+1) * batch_size

      # Grab the data and the labels for the batch
      X = x_validation[sidx:eidx]
      Y = y_validation[sidx:eidx]

      # Run the model
      Yhat = model(X)
      # Loss
      loss = -(torch.sum(Y*torch.log(Yhat+1e-15))/batch_size)

      # Keep track of the loss !!!
      validation_loss_np = loss.detach().cpu().numpy()
      new_validation_loss = new_validation_loss + validation_loss_np

      # Accuracy
      _, predicted = torch.max(Yhat.data,1)
      _, Y_validation = torch.max(Y,1)

      validation_total += Y_validation.size(0)
      validation_correct += (predicted == Y_validation).sum().item()


    new_validation_loss = new_validation_loss / n_validation
    validation_accuracy = (validation_correct / (n_batch_validation * batch_size)) * 100
    print(f'Validation: Epoch {epoch+1}/{n_epoch} {new_validation_loss:.5f} Loss, {validation_accuracy:.2f}% Accurate')
    validation_loss_coords.append(new_validation_loss)
    validation_acc_coords.append(validation_accuracy)
    if((new_validation_loss*.95) > prev_validation_loss) and (prev_validation_loss != 0):
        learningRate = learningRate/10
        print('Learning Rate Updated: ',learningRate)
    prev_validation_loss = new_validation_loss


# Test set
test_loss = 0.0
test_total = 0
test_correct = 0
images_pred = []
images_label = y_test[:50]

for batch in range(n_batch_test):
      # Start / end indicies of the data
      sidx = batch * batch_size
      eidx = (batch+1) * batch_size

      # Grab the data and the labels for the batch
      X_test = x_test[sidx:eidx]
      Y = y_test[sidx:eidx]

      # Run the model
      Yhat = model(X_test)
      # Loss
      loss = -(torch.sum(Y*torch.log(Yhat+1e-15))/batch_size)

      # Keep track of the loss !!!
      test_loss_np = loss.detach().cpu().numpy()
      test_loss = test_loss + test_loss_np

      # Accuracy
      _, predicted = torch.max(Yhat.data,1)
      _, Y_test = torch.max(Y,1)

      test_total += Y_test.size(0)
      test_correct += (predicted == Y_test).sum().item()

      # Show the first 10 images from the training set
      if(batch == 0):
        images_pred = predicted


test_loss = test_loss / n_test
test_accuracy = (test_correct / (n_batch_test * batch_size)) * 100
print('-----')
print(f'Test: Epoch {epoch+1}/{n_epoch} Training: {test_loss:.5f} Loss, {test_accuracy:.2f}% Accurate')
test_loss_coords.append(test_loss)
test_acc_coords.append(test_accuracy)

my_dict = dict([
    (0, 'airplane'),
    (1, 'automobile'),
    (2, 'bird'),
    (3, 'cat'),
    (4, 'deer'),
    (5, 'dog'),
    (6, 'frog'),
    (7, 'horse'),
    (8, 'ship'),
    (9, 'truck')])

for x in range(50):
  images_pred_np = images_pred[x].detach().cpu().numpy()
  images_label_np = torch.argmax(images_label[x]).detach().cpu().numpy()
  plt.figure()
  plt.imshow(x_test_np[x])
  # Add a title to the plot
  plt.title(f'Test: {x}  Label:<{my_dict[int(images_label_np)]}>  Pred:<{my_dict[int(images_pred_np)]}>  Accuracy: {test_accuracy:.2f}')

  # Save the figure as an image file (e.g., PNG)
  filename = os.path.join(outdir, f'test_{x}.png')
  plt.savefig(filename)
  matplotlib.pyplot.close()

# Create charts
filename = os.path.join(outdir, 'loss.png')
plt.savefig(filename)

fig1 = plt.figure()
plt.plot(list(range(len(train_loss_coords))), train_loss_coords, label='Train')
plt.plot(list(range(len(validation_loss_coords))), validation_loss_coords, label='Validation')
plt.axhline(y=test_loss_coords, label='Test', color='red')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('ResNet-3 Loss')
plt.legend()
plt.grid(True)
filename = os.path.join(outdir, 'loss.png')
plt.savefig(filename)

fig2 = plt.figure()
plt.plot(list(range(len(train_acc_coords))), train_acc_coords, label='Train')
plt.plot(list(range(len(validation_acc_coords))), validation_acc_coords, label='Validation')
plt.axhline(y=test_acc_coords, label='Test', color='red')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('ResNet-3 Accuracy')
plt.legend()
plt.grid(True)
filename = os.path.join(outdir, 'accuracy.png')
plt.savefig(filename)
matplotlib.pyplot.close()


100%|██████████| 170498071/170498071 [00:02<00:00, 61106519.84it/s]


Extracting ./content/cifar-10-python.tar.gz to ./content/
Files already downloaded and verified
Running on:  cuda
-----
Training: Epoch 1/100 0.01619 Loss, 41.84% Accurate
Validation: Epoch 1/100 0.01340 Loss, 51.40% Accurate
-----
Training: Epoch 2/100 0.01260 Loss, 54.62% Accurate
Validation: Epoch 2/100 0.01165 Loss, 58.14% Accurate
-----
Training: Epoch 3/100 0.01111 Loss, 60.21% Accurate
Validation: Epoch 3/100 0.01065 Loss, 61.70% Accurate
-----
Training: Epoch 4/100 0.01014 Loss, 63.92% Accurate
Validation: Epoch 4/100 0.00996 Loss, 64.00% Accurate
-----
Training: Epoch 5/100 0.00943 Loss, 66.46% Accurate
Validation: Epoch 5/100 0.00941 Loss, 66.56% Accurate
-----
Training: Epoch 6/100 0.00885 Loss, 68.59% Accurate
Validation: Epoch 6/100 0.00915 Loss, 67.54% Accurate
-----
Training: Epoch 7/100 0.00838 Loss, 70.31% Accurate
Validation: Epoch 7/100 0.00883 Loss, 68.62% Accurate
-----
Training: Epoch 8/100 0.00796 Loss, 71.83% Accurate
Validation: Epoch 8/100 0.00856 Loss, 69.78%